## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,46.40,57,75,33.38,broken clouds
1,1,Kapaa,US,22.0752,-159.3190,72.00,83,90,26.46,light rain
2,2,Rikitea,PF,-23.1203,-134.9692,80.40,74,39,8.46,scattered clouds
3,3,Bredasdorp,ZA,-34.5322,20.0403,64.40,94,13,3.44,few clouds
4,4,Karratha,AU,-20.7377,116.8463,81.28,62,31,12.68,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(15)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kapaa,US,22.0752,-159.3190,72.00,83,90,26.46,light rain
2,2,Rikitea,PF,-23.1203,-134.9692,80.40,74,39,8.46,scattered clouds
4,4,Karratha,AU,-20.7377,116.8463,81.28,62,31,12.68,scattered clouds
9,9,Richards Bay,ZA,-28.7830,32.0377,74.89,82,0,13.47,clear sky
11,11,Cidreira,BR,-30.1811,-50.2056,74.16,83,4,10.18,clear sky
12,12,Bathsheba,BB,13.2167,-59.5167,78.80,69,20,12.66,few clouds
13,13,Bhawanipatna,IN,19.9000,83.1667,71.29,45,0,3.91,clear sky
14,14,Yeppoon,AU,-23.1333,150.7333,78.80,69,40,5.75,scattered clouds
15,15,Hithadhoo,MV,-0.6000,73.0833,83.95,67,12,8.48,few clouds
16,16,Avarua,CK,-21.2078,-159.7750,84.20,70,62,9.22,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                301
City                   301
Country                299
Lat                    301
Lng                    301
Max Temp               301
Humidity               301
Cloudiness             301
Wind Speed             301
Current Description    301
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [8]:
clean_preferred_cities_df.count()

City_ID                299
City                   299
Country                299
Lat                    299
Lng                    299
Max Temp               299
Humidity               299
Cloudiness             299
Wind Speed             299
Current Description    299
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,72.00,light rain,22.0752,-159.3190,
2,Rikitea,PF,80.40,scattered clouds,-23.1203,-134.9692,
4,Karratha,AU,81.28,scattered clouds,-20.7377,116.8463,
9,Richards Bay,ZA,74.89,clear sky,-28.7830,32.0377,
11,Cidreira,BR,74.16,clear sky,-30.1811,-50.2056,
12,Bathsheba,BB,78.80,few clouds,13.2167,-59.5167,
13,Bhawanipatna,IN,71.29,clear sky,19.9000,83.1667,
14,Yeppoon,AU,78.80,scattered clouds,-23.1333,150.7333,
15,Hithadhoo,MV,83.95,few clouds,-0.6000,73.0833,
16,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,72.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
2,Rikitea,PF,80.40,scattered clouds,-23.1203,-134.9692,Pension Maro'i
4,Karratha,AU,81.28,scattered clouds,-20.7377,116.8463,ibis Styles Karratha
9,Richards Bay,ZA,74.89,clear sky,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
11,Cidreira,BR,74.16,clear sky,-30.1811,-50.2056,Hotel Castelo
12,Bathsheba,BB,78.80,few clouds,13.2167,-59.5167,Atlantis Hotel
13,Bhawanipatna,IN,71.29,clear sky,19.9000,83.1667,Bhagirathi inn
14,Yeppoon,AU,78.80,scattered clouds,-23.1333,150.7333,Villa Mar Colina
15,Hithadhoo,MV,83.95,few clouds,-0.6000,73.0833,Scoop Guest House
16,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,Paradise Inn


In [13]:
hotel_df.count()

City                   299
Country                299
Max Temp               299
Current Description    299
Lat                    299
Lng                    299
Hotel Name             299
dtype: int64

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   299
Country                299
Max Temp               299
Current Description    299
Lat                    299
Lng                    299
Hotel Name             299
dtype: int64

In [16]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,72.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
2,Rikitea,PF,80.40,scattered clouds,-23.1203,-134.9692,Pension Maro'i
4,Karratha,AU,81.28,scattered clouds,-20.7377,116.8463,ibis Styles Karratha
9,Richards Bay,ZA,74.89,clear sky,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
11,Cidreira,BR,74.16,clear sky,-30.1811,-50.2056,Hotel Castelo
12,Bathsheba,BB,78.80,few clouds,13.2167,-59.5167,Atlantis Hotel
13,Bhawanipatna,IN,71.29,clear sky,19.9000,83.1667,Bhagirathi inn
14,Yeppoon,AU,78.80,scattered clouds,-23.1333,150.7333,Villa Mar Colina
15,Hithadhoo,MV,83.95,few clouds,-0.6000,73.0833,Scoop Guest House
16,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,Paradise Inn


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))